In [1]:
import organize_files as of
import pandas as pd
import os
from sqlalchemy import create_engine
import datetime

In [2]:
db = 'AnzaBI'
db_user = 'db_Admin'
db_password = 'password'
db_host = '219.91.145.98'
db_port = '5432'
db_schema = '"Bowling_Data"'

try:
    engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db}',echo=True,future=True)
except:
    print("Unable to Create Database")
else:
    print("COnnection Successful")

COnnection Successful


In [3]:
if os.name == 'posix':
    file_loc = r"/home/shashankraj/Documents/DATA_Dump"
    log_file = file_loc+"/"+"Logs"
    final_files = file_loc+"/"+"Final_Df"
    backup = file_loc+"/"+"Backup"
    #today = str(datetime.today().date())
    trigger_file = file_loc+"/file_trigger/"+"new_data_received.txt"
else:
    file_loc=r"D:\One Drive Anza\OneDrive - Anza Services LLP\DATA_Dump\Final_Df"
    log_file = file_loc+"\\"+"Logs"
    final_files = file_loc+"\\"+"Final_Df"
    backup = file_loc+"\\"+"Backup"
    #today = str(datetime.today().date())
    trigger_file = file_loc+"\\file_trigger\\"+"new_data_received.txt"

In [4]:
schema_ref = {
    "Client Billing Descending":"client_billing","Fee Breakdown by Dept and Fee Earner":"fee_brkdn_dept_fe","Fee Summary by Dept and Fee Earner":"fee_smry_dept_fe",
    "Fees Billed":"fees_billed","Matter Source of Business inc Matter Bills (Bill Date)":"mttr_src_ref","Total Hours by Fee Earner-With Billings All":"tot_hrs_by_fe",
    "Matters Opened by FE":"mtrs_by_fe","Payment Received Analysis":"pmt_rcv_analysis"}

In [5]:
dict_list = of.categorize_files(file_loc)
all_files = of.concat_files(dict_list, file_loc, log_file)

print(f'\nLog Files are located at {log_file}')

Found 30 csv files

Log Files are located at /home/shashankraj/Documents/DATA_Dump/Logs


In [6]:
for file in all_files.keys():
        all_files[file].columns = [cols.lower() for cols in all_files[file].columns]
        fname = final_files+"/"+file+".csv"
        if os.path.exists(fname):
            print(f'{fname} already exists')
            all_files[file].to_csv(final_files+"/"+file+".csv",header=False,mode="a",index=False)
        else:
            print(f'Creating DF {fname}')
            all_files[file].to_csv(final_files+"/"+file+".csv",mode="a",index=False)
        
        try: 
            all_files[file].to_sql(schema_ref[file],con=engine,if_exists='append',schema='Bowling_Data',index=None)
        except:
            print(f"***** Unable to write to Database for file [{file}]********* ")
        else:
            print(f"Putting all data in Postgresql Server for  [{file}]")

/home/shashankraj/Documents/DATA_Dump/Final_Df/Client Billing Descending.csv already exists
2022-03-24 16:54:13,819 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2022-03-24 16:54:13,820 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-03-24 16:54:13,887 INFO sqlalchemy.engine.Engine select current_schema()
2022-03-24 16:54:13,888 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-03-24 16:54:13,963 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2022-03-24 16:54:13,964 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-03-24 16:54:14,030 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-03-24 16:54:14,031 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where n.nspname=%(schema)s and relname=%(name)s
2022-03-24 16:54:14,031 INFO sqlalchemy.engine.Engine [generated in 0.00115s] {'schema': 'Bowling_Data', 'name': 'client_billing'}
2022-03-24 16:54:14,099 INFO sqlalchemy.engine.Engine ROLLBACK
2022-03-24 16:54:1

In [7]:
for csv_file in os.listdir(final_files):
    print(f'Processing {csv_file}')
    if csv_file.endswith('.csv'):
        tmp_df = pd.read_csv(final_files+"/"+csv_file)
        tmp_df['date_added'] = pd.to_datetime(tmp_df['date_added'])
        fname = backup+"/"+csv_file.split(".")[0]+".parquet.gzip"
        print(f'Name is {fname}')
        if os.path.exists(fname):
            print(f'File already Exists, Removing')
            os.remove(fname)
        tmp_df.to_parquet(fname,compression = 'gzip')
        print("*"*50)

Processing Client Billing Descending.csv
Name is /home/shashankraj/Documents/DATA_Dump/Backup/Client Billing Descending.parquet.gzip
File already Exists, Removing
**************************************************
Processing Fee Breakdown by Dept and Fee Earner.csv
Name is /home/shashankraj/Documents/DATA_Dump/Backup/Fee Breakdown by Dept and Fee Earner.parquet.gzip
File already Exists, Removing
**************************************************
Processing Fee Summary by Dept and Fee Earner.csv
Name is /home/shashankraj/Documents/DATA_Dump/Backup/Fee Summary by Dept and Fee Earner.parquet.gzip
File already Exists, Removing
**************************************************
Processing Fees Billed.csv
Name is /home/shashankraj/Documents/DATA_Dump/Backup/Fees Billed.parquet.gzip
File already Exists, Removing
**************************************************
Processing Matter Source of Business inc Matter Bills (Bill Date).csv
Name is /home/shashankraj/Documents/DATA_Dump/Backup/Matter So

In [8]:
print("Performing Cleanup")

[os.remove(file_loc+"/Processed/"+file) for file in os.listdir(file_loc+"/Processed")]
[os.remove(file_loc+"/Unprocessed/"+file) for file in os.listdir(file_loc+"/Unprocessed")]
[os.remove(file_loc+"/file_trigger/"+file) for file in os.listdir(file_loc+"/file_trigger")]
[os.remove(file_loc+"/"+file) for file in os.listdir(file_loc) if file.endswith('.csv')]

Performing Cleanup


[]

In [9]:
# df_cb = all_files['Client Billing Descending']
# df_cb.head()
# "Cengiz Peri" and pd.Timestamp(datetime.date(2022,3,2)) in df_cb["date_added"].values

In [10]:
os.listdir(file_loc)

['Backup',
 'file_trigger',
 'Final_Df',
 'Logs',
 'Old Data Dump',
 'Processed',
 'Unprocessed']